In [1]:
from pyspark import SparkContext
import pyspark

conf = pyspark.SparkConf().set("spark.driver.host", "localhost")
sc = SparkContext.getOrCreate()
sc.setLogLevel('OFF')

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
21/11/18 12:30:29 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
21/11/18 12:30:29 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
supervisor_sf_pair = sc.textFile("../data/SF_business/supervisor_sf.tsv")\
                       .map(lambda x : x.split("\t"))\
                       .map(lambda x : (x[0], x[1]))
zip_biz_pair = sc.textFile("../data/SF_business/filtered_registered_business_sf.tsv")\
                 .map(lambda x : x.split("\t"))\
                 .map(lambda x : (x[0], x[1:]))

## 1. Using “filtered_registered_business_sf.tsv” and “supervisor_sf.tsv”, list unique business names without a supervisor ordered by name.

In [3]:
zip_biz_pair.subtractByKey(supervisor_sf_pair)\
            .values()\
            .map(lambda x : x[0])\
            .distinct()\
            .sortBy(lambda x : x)\
            .collect()

['"10up, Inc."',
 '"11 Main, Inc"',
 '"1490 Chestnut Street Associates, Llc"',
 '"1946 Washington Street, Llc"',
 '"360training.com, Inc."',
 '"5 Bars, Llc"',
 '"750 Brannan Street Properties, Llc"',
 '"7x7 Film, Llc"',
 '"8x8, Inc."',
 '"A Plus Tree, Inc."',
 '"A&e Television Networks, Llc"',
 '"A. K. Ngai Hawaii, Inc."',
 '"Abc Expo Services, Llc"',
 '"Abco Rest. & Waterproofing,inc"',
 '"Above All Plumbing, Inc"',
 '"Access Us Limo, Inc"',
 '"Acmii California 1, Llc"',
 '"Acq, Chad Feiner"',
 '"Acrt, Inc."',
 '"Act General Building Const,inc"',
 '"Acv Roofing, Inc Dba The Sullivan Group"',
 '"Adams & Smith, Inc."',
 '"Advance Construction Technology, Inc"',
 '"Advanced Geotechniques, Inc."',
 '"Advanced Roof Design, Inc."',
 '"Advanced Technology Distributors, Inc."',
 '"Advanced Traffic Products,inc."',
 '"Affinity Mobile, Llc"',
 '"Agci, Inc"',
 '"Agk Home Solutions, Inc"',
 '"Ahmed, Alakwaa"',
 '"Airespring, Inc"',
 '"Airport Terminal Services, Inc"',
 '"Al-Tar Services, Inc."',


## 2. Create pairs of business name and supervisor id ordered by supervisor id (first) and business name (second).

In [4]:
zip_biz_pair.join(supervisor_sf_pair)\
            .values()\
            .map(lambda x : (x[0][0], x[1]))\
            .distinct()\
            .sortBy(lambda x : (x[1], x[0]))\
            .collect()

[('"1125 Sir Francis Drake Boulevard Operating Company, Llc"', '1'),
 ('"1909 Fillmore, Inc"', '1'),
 ('"2700 Geary Partners, L.p."', '1'),
 ('"2705 Jackson Street, Dba"', '1'),
 ('"2bkco, Inc."', '1'),
 ('"303 Austin Street, Llc"', '1'),
 ('"4 Event Design, Llc"', '1'),
 ('"49 North, Llc"', '1'),
 ('"525,655 Hyde Street Cnml Properties, Llc"', '1'),
 ('"A & E Properties, Llc"', '1'),
 ('"A.m.p. & Associates, Llc"', '1'),
 ('"Abuali, Najwa"', '1'),
 ('"Adapt : A Behavioral Collective, Llc"', '1'),
 ('"Adapt: A Behavioral Collective, Inc."', '1'),
 ('"Adh Properties, Llc"', '1'),
 ('"Akin, Inc."', '1'),
 ('"Alaya, Inc."', '1'),
 ('"Alexander Vilderman Dds, Inc."', '1'),
 ('"Amireh Enterprises, Inc"', '1'),
 ('"Anakeysar Capital, Llc"', '1'),
 ('"Appraisal Connections, Inc"', '1'),
 ('"Aquino Md, Melinda L"', '1'),
 ('"Aramark Facility Services, Llc"', '1'),
 ('"Arch Construction, Inc."', '1'),
 ('"Avignon Julian, Llc"', '1'),
 ('"Batchelor Holdings, Llc"', '1'),
 ('"Bay Area Ip, Llc"', 

## 3. Create pairs of business and supervisor id for all the business ordered by supervisor id (first) and business name (second). (although it may not have a district supervisor).

In [5]:
def return_name_id(x):
    if x[1] == None:
        return (x[0][0], '')
    else: 
        return (x[0][0], x[1])

In [6]:
zip_biz_pair.leftOuterJoin(supervisor_sf_pair)\
            .values()\
            .map(return_name_id)\
            .distinct()\
            .sortBy(lambda x : (x[1], x[0]))\
            .collect()

[('"10up, Inc."', ''),
 ('"11 Main, Inc"', ''),
 ('"1490 Chestnut Street Associates, Llc"', ''),
 ('"1946 Washington Street, Llc"', ''),
 ('"360training.com, Inc."', ''),
 ('"5 Bars, Llc"', ''),
 ('"750 Brannan Street Properties, Llc"', ''),
 ('"7x7 Film, Llc"', ''),
 ('"8x8, Inc."', ''),
 ('"A Plus Tree, Inc."', ''),
 ('"A&e Television Networks, Llc"', ''),
 ('"A. K. Ngai Hawaii, Inc."', ''),
 ('"Abc Expo Services, Llc"', ''),
 ('"Abco Rest. & Waterproofing,inc"', ''),
 ('"Above All Plumbing, Inc"', ''),
 ('"Access Us Limo, Inc"', ''),
 ('"Acmii California 1, Llc"', ''),
 ('"Acq, Chad Feiner"', ''),
 ('"Acrt, Inc."', ''),
 ('"Act General Building Const,inc"', ''),
 ('"Acv Roofing, Inc Dba The Sullivan Group"', ''),
 ('"Adams & Smith, Inc."', ''),
 ('"Advance Construction Technology, Inc"', ''),
 ('"Advanced Geotechniques, Inc."', ''),
 ('"Advanced Roof Design, Inc."', ''),
 ('"Advanced Technology Distributors, Inc."', ''),
 ('"Advanced Traffic Products,inc."', ''),
 ('"Affinity Mobile

In [7]:
sc.stop()